# WORKING WITH API TO DATAFRAME:

In [9]:
import time
import requests
import pandas as pd

In [10]:
response = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page=1')

In [11]:
temp_df = pd.DataFrame(response.json()['results'])[['id', 'title', 'overview', 'release_date', 'popularity', 'vote_average', 'vote_count']]

In [12]:
temp_df.sample(5)

,id,title,overview,release_date,popularity,vote_average,vote_count
10,680,Pulp Fiction,"A burger-loving hit man, his philosophical par...",1994-09-10,106.045,8.500,28110
14,429,"The Good, the Bad and the Ugly",While the Civil War rages on between the Union...,1966-12-22,89.184,8.500,8707
16,769,GoodFellas,"The true story of Henry Hill, a half-Irish, ha...",1990-09-12,76.604,8.457,12982
12,122,The Lord of the Rings: The Return of the King,As armies mass for a final battle that will de...,2003-12-17,185.538,8.483,24426
9,496243,Parasite,"All unemployed, Ki-taek's family takes peculia...",2019-05-30,110.368,8.500,18450


In [13]:
df = pd.DataFrame()

In [14]:
df

""


In [15]:
# Session for connection reuse
with requests.Session() as session:
    for i in range(1, 495):
        
        # Maximum number of retries
        retries = 5  
        success = False

        for attempt in range(retries):
            try:
                # Send the GET request
                response = session.get(
                    f'https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={i}',
                    timeout=10
                )
                response.raise_for_status()  

                # Process the response data
                temp_df = pd.DataFrame(response.json()['results'])[
                    ['id', 'title', 'overview', 'release_date', 'popularity', 'vote_average', 'vote_count']
                ]
                
                df = pd.concat([df, temp_df], ignore_index=True)
                
                # print(f"Page {i} fetched successfully.")
                success = True
                
                
                # Exit the retry loop on success
                break  
            
            
            except requests.exceptions.RequestException as e:
                print(f"Error on page {i}, attempt {attempt + 1}: {e}")
                
                # Exponential backoff: 1s, 2s, 4s, etc.
                time.sleep(2 ** attempt)  



        if not success:
            print(f"Failed to fetch page {i} after {retries} retries.")

In [16]:
df

,id,title,overview,release_date,popularity,vote_average,vote_count
0,278,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,1994-09-23,186.654,8.700,27442
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,176.436,8.690,20829
2,240,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,95.884,8.572,12573
3,424,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,91.333,8.567,15996
4,389,12 Angry Men,The defense and the prosecution have rested an...,1957-04-10,61.916,8.500,8772
...,...,...,...,...,...,...,...
9866,12142,Alone in the Dark,Edward Carnby is a private investigator specia...,2005-01-28,11.939,3.200,609
9867,13805,Disaster Movie,"Over the course of one evening, an unsuspectin...",2008-08-29,17.121,3.200,1034
9868,11059,House of the Dead,"Set on an island off the coast, a techno rave ...",2003-04-11,9.899,3.100,389
9869,14164,Dragonball Evolution,"On his 18th birthday, Goku receives a mystical...",2009-03-12,22.022,2.900,2051


In [17]:
df.shape

(9871, 7)

In [18]:
df.to_csv('movies.csv')